In [ ]:
import json
import pandas as pd
from collections import defaultdict
import requests
import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon

In [ ]:
#read json file
filename = "PM.json"
output = 'lga_site.json'
with open(filename, "r") as file:
    pm = json.load(file)

In [ ]:
url = "https://gateway.api.epa.vic.gov.au/environmentMonitoring/v1/sites?environmentalSegment=air"

headers = {
    "User-Agent": 'curl/8.4.0',
    "Cache-Control": 'no-cache',
    "X-API-Key": "62eb03b279a54273ace3f893e994f90d"
}

In [ ]:
try:
    response = requests.get(url, headers=headers)
    raw_epa = response.json()
except Exception as e:
    print("error when request EPA data: ", e)

In [ ]:
site = defaultdict()
for record in raw_epa['records']:
    site[record['siteName']] = record['geometry']['coordinates']

In [ ]:
with open('VIC_LGA_2021.geojson', "r") as geo_file:
    geo = json.load(geo_file)['features']

In [ ]:
lga_site = defaultdict()
n = 0
for record in geo:
    name = record['properties']['lga_name'][0]
    print('LGA: '+name)
    shape = shapely.from_geojson(json.dumps(record['geometry']))
    for i in site.keys():
        print(i)
        inArea = shape.contains(Point(site[i][1],site[i][0]))
        if inArea:
            n += 1
            if name in lga_site.keys():
                lga_site[name].append(i)
            else: lga_site[name] = [i]
            print('matched')

In [ ]:
with open(output,'w') as out:
    json.dump(lga_site,out,indent=2)

In [6]:
with open('VIC_LGA_2021.geojson', "r") as geo_file:
    geodata = json.load(geo_file)
for feature in geodata.get("features", []):
    lga_name = feature["properties"].get("lga_name")[0]
    feature["properties"]["LGA"] = lga_name


In [7]:
with open('LGA.geojson','w') as out:
    json.dump(geodata,out,indent=2)